### Análisis Exploratorio

Instalar y cargar librerías

In [48]:
! pip install surprise

  Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for scikit-surprise: started
  Running setup.py install for scikit-surprise: finished with status 'error'


  DEPRECATION: scikit-surprise is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  error: subprocess-exited-with-error
  
  × Running setup.py install for scikit-surprise did not run successfully.
  │ exit code: 1
  ╰─> [78 lines of output]
      running install
      C:\Python311\Lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\surprise
      copying surprise\accuracy.py -> build\lib.win-amd64-cpython-311\surprise
      copy

In [49]:
import numpy as np
import pandas as pd 
import sqlite3 as sql
import plotly.express as px
import funciones as fn

Análisis

In [50]:
#Realizar conexión de base de datos db_movies
conn=sql.connect('db_movies')
cur=conn.cursor()

In [51]:
#Verificar las tablas existentes en db_movies
cur.execute("SELECT name FROM sqlite_master where type='table' ")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

In [52]:
#Cargar tablas de db_movies
ratings=pd.read_sql("""select * from ratings""", conn)
movies=pd.read_sql("""select * from movies""", conn)

In [53]:
#Datos no nulos, tipo de variable, cantidad de registros y columnas de tabla 'movies'
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [54]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [55]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [56]:
#Datos no nulos, tipo de variable, cantidad de registros y columnas de tabla 'ratings'
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [57]:
#Calcular la distribución de calificaciones de tabla 'ratings'
cr=pd.read_sql("""select 
                        rating,
                        count(*) as conteo
                        from ratings
                        group by rating
                        order by conteo desc""", conn)

In [58]:
# crear gráfica
fig = px.bar(cr, x = cr.rating, y=cr.conteo, 
             title= '<b>Conteo de rating<b>', text_auto=True,
             color_discrete_sequence=px.colors.qualitative.Pastel1)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Rating',
    yaxis_title = 'Conteo',
    template = 'simple_white',
    title_x = 0.5)

fig.update_traces(marker_line_color='#000000',
                  marker_line_width=2)

fig.show()

In [59]:
#Calcular cada usuario cuántas peliculas calificó
rating_users=pd.read_sql('''select UserId,
                        count(*) as cnt_rat
                        from ratings
                        group by UserId
                        order by cnt_rat asc
                        ''', conn)

In [60]:
rating_users.head()


,userId,cnt_rat
0,53,20
1,147,20
2,189,20
3,194,20
4,207,20


In [61]:
# crear gráfica
fig = px.histogram(rating_users, x = rating_users.cnt_rat, 
             title= '<b>Histograma frecuencia de número de calificaciones por usuario<b>', text_auto=True,
             color_discrete_sequence=px.colors.qualitative.Pastel1)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Películas calificadas',
    yaxis_title = 'Cantidad',
    template = 'simple_white',
    title_x = 0.5)

fig.update_traces(marker_line_color='#000000',
                  marker_line_width=2)

fig.show()

In [62]:
rating_users.describe() 

,userId,cnt_rat
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


In [63]:
#Filtrar usuarios con 35 o más calificaciones
rating_users2=pd.read_sql('''select UserId,
                        count(*) as cnt_rat
                        from ratings
                        group by UserId
                        having cnt_rat>=35
                        order by cnt_rat desc
                        ''', conn)

In [64]:
rating_users2.describe()

,userId,cnt_rat
count,465.000000,465.000000
mean,303.339785,208.739785
std,177.663222,295.556108
min,1.000000,35.000000
25%,149.000000,56.000000
50%,305.000000,109.000000
75%,453.000000,226.000000
max,610.000000,2698.000000


In [65]:
# crear gráfica
fig = px.histogram(rating_users2, x = rating_users.cnt_rat, 
             title= '<b>Histograma frecuencia de número de calificaciones por usuario<b>', text_auto=True,
             color_discrete_sequence=px.colors.qualitative.Pastel1)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Películas calificadas',
    yaxis_title = 'Cantidad',
    template = 'simple_white',
    title_x = 0.5)

fig.update_traces(marker_line_color='#000000',
                  marker_line_width=2)

fig.show()

In [66]:
#Verificar cuantas calificaciones tiene cada película
rating_movie=pd.read_sql('''select movieId,
                        count(*) as cnt_rat
                        from ratings
                        group by movieId
                        order by cnt_rat desc
                        ''', conn)

In [67]:
rating_movie.head()

,movieId,cnt_rat
0,356,329
1,318,317
2,296,307
3,593,279
4,2571,278


In [68]:
rating_movie.describe()

,movieId,cnt_rat
count,9724.000000,9724.000000
mean,42245.024373,10.369807
std,52191.137320,22.401005
min,1.000000,1.000000
25%,3245.500000,1.000000
50%,7300.000000,3.000000
75%,76739.250000,9.000000
max,193609.000000,329.000000


In [69]:
# crear gráfica
fig = px.histogram(rating_movie, x = rating_movie.cnt_rat, 
             title= '<b>Histograma de frecuencia de número de calificaciones por película<b>', text_auto=True,
             color_discrete_sequence=px.colors.qualitative.Pastel1)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Rating',
    yaxis_title = 'Count',
    template = 'simple_white',
    title_x = 0.5)

fig.update_traces(marker_line_color='#000000',
                  marker_line_width=2)

fig.show()

In [70]:
#Filtrar películas con 9 o más calificaciones
rating_movie2=pd.read_sql('''select movieId,
                        count(*) as cnt_rat
                        from ratings
                        group by movieId
                        having cnt_rat>=9
                        order by cnt_rat desc
                        ''', conn)

In [71]:
rating_movie2.describe()

,movieId,cnt_rat
count,2441.000000,2441.000000
mean,20878.934453,33.864810
std,35509.052528,35.364643
min,1.000000,9.000000
25%,1379.000000,13.000000
50%,3316.000000,21.000000
75%,8984.000000,41.000000
max,187593.000000,329.000000


In [72]:
# crear gráfica
fig = px.histogram(rating_movie2, x = rating_movie2.cnt_rat, 
             title= '<b>Histograma de frecuencia de número de calificaciones por película<b>', text_auto=True,
             color_discrete_sequence=px.colors.qualitative.Pastel1)

# agregar detalles a la gráfica
fig.update_layout(
    xaxis_title = 'Rating',
    yaxis_title = 'Count',
    template = 'simple_white',
    title_x = 0.5)

fig.update_traces(marker_line_color='#000000',
                  marker_line_width=2)

fig.show()

In [73]:
fn.ejecutar('Preprocesamiento.sql', cur)

cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

In [74]:
### verficar tamaño de tablas con filtros ####

## ratings
print(pd.read_sql('select count(*) from ratings', conn))
print(pd.read_sql('select count(*) from ratings_final', conn))


   count(*)
0    100836
   count(*)
0     79157


In [75]:
## movies
print(pd.read_sql('select count(*) from movies', conn))
print(pd.read_sql('select count(*) from movies_sel', conn))

   count(*)
0      9742
   count(*)
0      2441


In [76]:
## 2 tablas cruzadas ###
pd.read_sql('select count(*) from full_ratings', conn)

,count(*)
0,79157


In [80]:
ratingsf=pd.read_sql('select * from full_ratings',conn)

In [81]:
ratingsf.head()

,userId,rating,movieId,title,genres
0,1,4.0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,4.0,3,Grumpier Old Men (1995),Comedy|Romance
2,1,4.0,6,Heat (1995),Action|Crime|Thriller
3,1,5.0,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,5.0,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [82]:
#Verificar si hay presencia de duplicados
ratingsf.duplicated().sum()

0

### Recomendaciones basado en popularidad

In [90]:
#### Peliculas mejores calificadas
pd.read_sql("""select title, 
            avg(rating) as avg_rat,
            count(*) as cal_num
            from full_ratings
            group by title
            order by avg_rat desc
            limit 10
            
            """, conn)


,title,avg_rat,cal_num
0,Secrets & Lies (1996),4.590909,11
1,Guess Who's Coming to Dinner (1967),4.545455,11
2,Paths of Glory (1957),4.541667,12
3,"Streetcar Named Desire, A (1951)",4.470588,17
4,"Celebration, The (Festen) (1998)",4.458333,12
5,"Shawshank Redemption, The (1994)",4.438182,275
6,Ran (1985),4.433333,15
7,His Girl Friday (1940),4.392857,14
8,Witness for the Prosecution (1957),4.388889,9
9,Lawrence of Arabia (1962),4.360465,43


In [87]:
#### Películas más vistas###
pd.read_sql("""select title, 
            avg(rating) as avg_rat,
            count(*) as view_num
            from full_ratings
            group by title
            order by view_num desc
            """, conn)

,title,avg_rat,view_num
0,Forrest Gump (1994),4.153716,296
1,Pulp Fiction (1994),4.206835,278
2,"Shawshank Redemption, The (1994)",4.438182,275
3,"Silence of the Lambs, The (1991)",4.190476,252
4,"Matrix, The (1999)",4.161943,247
...,...,...,...
2436,Blue Streak (1999),2.562500,8
2437,Antwone Fisher (2002),3.375000,8
2438,Anna and the King (1999),3.562500,8
2439,"Absent-Minded Professor, The (1961)",3.437500,8
